In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import date, timedelta, datetime
from fake_useragent import UserAgent
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError, RequestException

C:\Users\oksik\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
def get_links(url, headers):
    try:
        time.sleep(1)
        page=requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, "html.parser")
        soup2 = soup.find_all('li')
        links = ['https://www.tadviser.ru'+ s.find('a')['href'] for s in soup2]
        titles = [s.find('a').text for s in soup2]
        return pd.DataFrame({'link': links,'title': titles})
    except requests.ConnectionError as e:
        print(e)
        log_error(e)
    except requests.Timeout as e:
        print(e)
        log_error(e)
    except requests.RequestException as e:
        print(e)
        log_error(e)
    except requests.ConnectTimeout as e:
        print(e)
        log_error(e)
    except requests.ReadTimeout as e:
        print(e)
        log_error(e)
    except requests.HTTPError as e:
        print(e)
        log_error(e) 

def get_text(url, headers):
    try:
        time.sleep(1)
        articles = requests.get(url, headers=headers)
        articles_soup = BeautifulSoup(articles.text, "html.parser")
        articles_content = articles_soup.find_all('p')[2:]
        articles_text = [paragraph.get_text() for paragraph in articles_content]
        return ' '.join(articles_text)
    except requests.ConnectionError as e:
        print(e)
        log_error(e)
    except requests.Timeout as e:
        print(e)
        log_error(e)
    except requests.RequestException as e:
        print(e)
        log_error(e)
    except requests.ConnectTimeout as e:
        print(e)
        log_error(e)
    except requests.ReadTimeout as e:
        print(e)
        log_error(e)
    except requests.HTTPError as e:
        print(e)
        log_error(e)
    
def log_error(error):
    pass

ua = UserAgent()
headers = {'accept': '*/*', 'user-agent': ua.random}
today = date.today()
art_date = today.strftime("%d.%m.%Y")
df_tadviser = pd.DataFrame({'link': [], 'title': [], 'text': [], 'date': []})

while art_date != '01.01.2021':
    if art_date[3] == '0':
        art_date = art_date[:3]+art_date[4:]
    if art_date[0] == '0':
        art_date = art_date[1:]   
    url = 'https://www.tadviser.ru/index.php/Архив_новостей?cdate='+ art_date
    df = get_links(url, headers)
    content = [get_text(url, headers) for url in df['link'].tolist()]
    df['text'] = content
    df['date'] = pd.to_datetime(today).strftime('%d.%m.%Y')
    df_tadviser = pd.concat([df_tadviser, df], axis=0)
    print(art_date)
    today -= timedelta(days=1)
    art_date = today.strftime("%d.%m.%Y")

In [69]:
df_tadviser = df_tadviser.replace(r'\t|\n|\r|\s+',' ', regex=True)
for col in df_tadviser.select_dtypes('object'):
    df_tadviser[col] = df_tadviser[col].str.strip()
df_tadviser = df_tadviser[df_tadviser['text'].astype(bool)]
df_tadviser['source']='Tadviser'
df_tadviser

In [76]:
df_tadviser.to_csv(f'C:\\Users\\oksik\\Desktop\\Tadviser_done.csv', index=False, sep='\t', 
             encoding='utf-8', header=True)